In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
train = pd.read_csv('train.csv')
test = pd.read_csv('test_x.csv')

Y_train = train['per_square_meter_price']
train = train.drop(columns = ['per_square_meter_price'],axis = 1)

In [2]:
#Заполнение пропусков
train['floor'] = train['floor'].fillna(0)
test['floor'] = test['floor'].fillna(0)
train['street'] = train['street'].fillna('0')
test['street'] = test['street'].fillna('0')
train['osm_city_nearest_population'] = train['osm_city_nearest_population'].fillna(0)
test['osm_city_nearest_population'] = test['osm_city_nearest_population'].fillna(0)

train_fil = train[['reform_house_population_1000','reform_house_population_500','reform_mean_floor_count_1000','reform_mean_floor_count_500','reform_mean_year_building_1000','reform_mean_year_building_500']]
train_fil = train_fil.fillna(train_fil.mean())
test_fil = test[['reform_house_population_1000','reform_house_population_500','reform_mean_floor_count_1000','reform_mean_floor_count_500','reform_mean_year_building_1000','reform_mean_year_building_500']]
test_fil = test_fil.fillna(test_fil.mean())
train[['reform_house_population_1000','reform_house_population_500','reform_mean_floor_count_1000','reform_mean_floor_count_500','reform_mean_year_building_1000','reform_mean_year_building_500']] = train_fil
test[['reform_house_population_1000','reform_house_population_500','reform_mean_floor_count_1000','reform_mean_floor_count_500','reform_mean_year_building_1000','reform_mean_year_building_500']] = test_fil

In [3]:
#Работа со столбцом street, убираю столбцы date,city,id
train['street'] = train['street'].replace(to_replace ='S', value = '', regex = True)
test['street'] = test['street'].replace(to_replace ='S', value = '', regex = True)
train['street'] = train['street'].astype(int)
test['street'] = test['street'].astype(int)

train = train.drop(columns = ['date'],axis = 1)
train = train.drop(columns = ['id'],axis = 1)
train = train.drop(columns = ['city'],axis = 1)

test = test.drop(columns = ['date'],axis = 1)
test = test.drop(columns = ['id'],axis = 1)
test = test.drop(columns = ['city'],axis = 1)

In [4]:
#Работа с object
train_cat = train[['region']]
train_cat = train_cat.merge(pd.get_dummies(train_cat.region, drop_first=True), left_index=True, right_index=True)
train_cat.drop('region', axis = 1, inplace=True)

test_cat = test[['region']]
test_cat = test_cat.merge(pd.get_dummies(test_cat.region, drop_first=True), left_index=True, right_index=True)
test_cat.drop('region', axis = 1, inplace=True)



In [5]:
#Работа с числовыми столбцами
train_numeric = train.drop(['region','osm_city_nearest_name'], axis = 1)
train_numeric_cols = train_numeric.columns
test_numeric = test.drop(['region','osm_city_nearest_name'], axis = 1)
test_numeric_cols = test_numeric.columns

from sklearn.preprocessing import StandardScaler
from scipy import stats
ss = StandardScaler()
train_numeric = pd.DataFrame(ss.fit_transform(train_numeric))
train_numeric.columns = train_numeric_cols
test_numeric = pd.DataFrame(ss.fit_transform(test_numeric))
test_numeric.columns = test_numeric_cols

In [6]:
#Слить таблицы
train = pd.merge(train_numeric, train_cat, left_index = True, right_index = True)
test = pd.merge( test_numeric, test_cat, left_index = True, right_index = True)


In [7]:
from sklearn.feature_selection import SelectKBest, f_regression
X_reg=train
y_reg=Y_train

kb = SelectKBest(score_func=f_regression, k=45)
kb.fit(X_reg, y_reg)
features_list = X_reg.columns
chosen_list = features_list[kb.get_support()]
print(features_list[kb.get_support()])

Index(['lng', 'osm_amenity_points_in_0.001', 'osm_amenity_points_in_0.005',
       'osm_amenity_points_in_0.0075', 'osm_amenity_points_in_0.01',
       'osm_catering_points_in_0.001', 'osm_catering_points_in_0.005',
       'osm_catering_points_in_0.0075', 'osm_catering_points_in_0.01',
       'osm_city_nearest_population', 'osm_crossing_points_in_0.005',
       'osm_crossing_points_in_0.0075', 'osm_crossing_points_in_0.01',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.005',
       'osm_finance_points_in_0.0075', 'osm_finance_points_in_0.01',
       'osm_healthcare_points_in_0.005', 'osm_healthcare_points_in_0.0075',
       'osm_healthcare_points_in_0.01', 'osm_historic_points_in_0.005',
       'osm_historic_points_in_0.0075', 'osm_historic_points_in_0.01',
       'osm_hotels_points_in_0.005', 'osm_hotels_points_in_0.0075',
       'osm_hotels_points_in_0.01', 'osm_leisure_points_in_0.005',
       'osm

In [8]:
#Превращаем в массивы
y_train = Y_train.values
train = train[chosen_list]
x_train = train.values
test = test[chosen_list]
x_test = test.values


# Делим на тренировочную и тестовую
from sklearn.model_selection import train_test_split
x_t1,x_t2,y_t1,y_t2 = train_test_split(x_train, y_train, test_size=0.7, random_state=0)


In [9]:
from sklearn.linear_model import LinearRegression
ml = LinearRegression()
ml.fit(x_t1, y_t1)
y_pred=ml.predict(x_t2)
y_pred_test=ml.predict(x_test)
print(y_pred)
print(y_pred_test)

[ 35274.83353826 315583.13040515  44368.15812337 ...  93308.60447989
 417251.63630865 129459.65087897]
[215297.73127696  32731.26940606  56683.3944445  ... 150422.93322316
 208828.34781288  52048.08189579]


In [10]:
#оценка погрешности
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print(mean_absolute_percentage_error(y_t2,y_pred))
from sklearn.metrics import r2_score
print(r2_score(y_t2,y_pred))

222.64043811647997
0.4544355215791551


In [11]:
test['per_square_meter_price'] = y_pred_test/3

In [12]:
answer = test['per_square_meter_price']
answer.to_csv('lab4.csv')